In [1]:
import pandas as pd 
lgt_csv = pd.read_csv('lgt_AO(2019-2025).csv')
lgt = lgt_csv['0'] 

i=0
c=100

newdf = pd.DataFrame(columns=['c1'], index=range(21))
while i<=20:
    c = (c*((lgt[i]-100) * 1.4575111432282974+100))/100
    newdf.loc[i].c1 = c
    i=i+1 
ltrnS=newdf['c1']
l = newdf['c1']
trn_l = l.shift(1)
trn_l[0] = 100

ltrn = ltrnS/trn_l*100

ltrn

0     103.644
1     101.593
2     100.501
3     100.371
4     100.992
5     98.9877
6     99.2233
7         NaN
8         NaN
9         NaN
10        NaN
11        NaN
12        NaN
13        NaN
14        NaN
15        NaN
16        NaN
17        NaN
18        NaN
19        NaN
20        NaN
Name: c1, dtype: object